# Working with the EIA Extract / Transform
This notebook steps through PUDL's extract and transform steps for the EIA 860 and 923 datasets, to make it easier to test and add new years of data, or new tables from the various spreadsheets that haven't been integrated yet.

In [ ]:
%load_ext autoreload
%autoreload 2
import pudl
import logging
import sys
from pathlib import Path
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()

## Set the scope for the Extract-Transform:

In [ ]:
from pudl.settings import Eia860Settings, Eia923Settings, EiaSettings
from pudl.metadata.classes import DataSource

eia860_data_source = DataSource.from_id("eia860")
eia860_years = eia860_data_source.working_partitions["years"]
#eia860_years = [2020]
eia860_settings = Eia860Settings(years=eia860_years)

# Uncomment to use all available years:
eia923_data_source = DataSource.from_id("eia923")
eia923_years = eia923_data_source.working_partitions["years"]
#eia923_years = [2020]
eia923_settings = Eia923Settings(years=eia923_years)

eia_settings = EiaSettings(eia860=eia860_settings, eia923=eia923_settings)

## Create a locally cached datastore

In [ ]:
ds_kwargs = {"local_cache_path": pudl_settings["data_dir"]}
ds = pudl.workspace.datastore.Datastore(**ds_kwargs)

# EIA-860

## Extract just the EIA-860 / EIA-860m

In [ ]:
%%time
eia860_extractor = pudl.extract.eia860.Extractor(ds)
eia860_raw_dfs = eia860_extractor.extract(year=eia860_settings.years)
if eia860_settings.eia860m:
    eia860m_raw_dfs = pudl.extract.eia860m.Extractor(ds).extract(
        year_month=eia860_settings.eia860m_date
    )
    eia860_raw_dfs = pudl.extract.eia860m.append_eia860m(
        eia860_raw_dfs=eia860_raw_dfs,
        eia860m_raw_dfs=eia860m_raw_dfs
    )

## Transform EIA-860 / EIA-860m

In [ ]:
%%time
eia860_transformed_dfs = pudl.transform.eia860.transform(
    eia860_raw_dfs,
    eia860_tables=eia860_settings.tables,
)

# EIA-923

## Extract just the EIA-923

In [ ]:
%%time
eia923_extractor = pudl.extract.eia923.Extractor(ds)
eia923_raw_dfs = eia923_extractor.extract(year=eia923_settings.years)

## Transform just the EIA-923

In [ ]:
%%time
eia923_transformed_dfs = pudl.transform.eia923.transform(
    eia923_raw_dfs,
    eia923_tables=eia923_settings.tables,
)

# Combined EIA Data

## Merge EIA-923/860, set dtypes, harvest entities

In [ ]:
%%time
eia_transformed_dfs = eia923_transformed_dfs.copy()
eia_transformed_dfs.update(eia860_transformed_dfs.copy())

# Do some final cleanup and assign appropriate types:
eia_transformed_dfs = {
    name: pudl.helpers.convert_cols_dtypes(df, data_source="eia")
    for name, df in eia_transformed_dfs.items()
}
    
entities_dfs, eia_transformed_dfs = pudl.transform.eia.transform(
    eia_transformed_dfs,
    eia860_years=eia860_settings.years,
    eia923_years=eia923_settings.years,
    eia860m=eia860_settings.eia860m,
)

# Assign appropriate types to new entity tables:
entities_dfs = {
    name: pudl.helpers.apply_pudl_dtypes(df, group="eia")
    for name, df in entities_dfs.items()
}

for table in entities_dfs:
    entities_dfs[table] = (
        pudl.metadata.classes.Package.from_resource_ids()
        .get_resource(table)
        .encode(entities_dfs[table])
    )

out_dfs = pudl.etl._read_static_tables_eia()
out_dfs.update(entities_dfs)
out_dfs.update(eia_transformed_dfs)